In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def mnist_binary(images, labels, binary):
#     binary = [binary[0]-1 , binary[1]-1]
    select = [i for i,l in enumerate(labels) if sum(l[binary])>=1]
    x = images[select]
    y = labels[select][:,binary].astype(int)
    return x, y

In [4]:
numbers = [1, 2, 3, 4, 5]
train_x, train_y = mnist_binary(mnist.train.images, mnist.train.labels, numbers)
test_x, test_y = mnist_binary(mnist.test.images, mnist.test.labels, numbers)

In [5]:
len_numbers = len(numbers)

In [6]:
def variable_summaries(var, name):
  """Attach a lot of summaries to a Tensor."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

In [8]:
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [9]:
  def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights, layer_name + '/weights')
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases, layer_name + '/biases')
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.histogram_summary(layer_name + '/pre_activations', preactivate)
      activations = act(preactivate, 'activation')
      tf.histogram_summary(layer_name + '/activations', activations)
      return activations

In [10]:
# set variables
with tf.name_scope('input'):
  x = tf.placeholder(tf.float32, [None, 784], name='x-input') # 28*28
  y_ = tf.placeholder(tf.float32, [None, len_numbers], name='y-input')

In [11]:
with tf.name_scope('input_reshape'):
  image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
  tf.image_summary('input', image_shaped_input, 10) # 10 sample in IMAGES of tensorboard 

In [12]:
# Set graphs and GRAPHS of tensorboard
y = nn_layer(x, 784, len_numbers, 'layer1', tf.nn.softmax)

In [13]:
# Set loss function
with tf.name_scope('cross_entropy'):
  diff = y_ * tf.log(y)
  with tf.name_scope('total'):
    cross_entropy = -tf.reduce_mean(diff)
  tf.scalar_summary('cross entropy', cross_entropy)

In [14]:
# Set optimizer method
with tf.variable_scope("trainer") as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    trainer = optimizer.minimize(cross_entropy)

In [15]:
with tf.name_scope('accuracy'):
  with tf.name_scope('correct_prediction'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.scalar_summary('accuracy', accuracy)

In [16]:
merged = tf.merge_all_summaries()

In [17]:
sess = tf.Session()

In [18]:
train_writer = tf.train.SummaryWriter('mnist_logs' + '/train', sess.graph)
test_writer = tf.train.SummaryWriter('mnist_logs' + '/test', sess.graph)

In [19]:
sess.run(tf.initialize_all_variables())

In [20]:
for i in range(1000): # train 1000 times
    sess.run(trainer, feed_dict={x: train_x, y_: train_y})
    result = sess.run(merged, feed_dict={x: train_x, y_: train_y})
    train_writer.add_summary(result, i)
    if i % 50 == 0: # save summary if after train 50 times
        result, acc = sess.run([merged, accuracy], feed_dict={x: test_x, y_: test_y})
        test_writer.add_summary(result, i)
        print "accuracy is %0.8s at times %4d" % (acc,i)

accuracy is 0.250247 at times    0
accuracy is 0.305682 at times   50
accuracy is 0.422491 at times  100
accuracy is 0.523461 at times  150
accuracy is 0.617699 at times  200
accuracy is 0.684815 at times  250
accuracy is 0.723025 at times  300
accuracy is 0.749753 at times  350
accuracy is 0.773114 at times  400
accuracy is 0.788161 at times  450
accuracy is 0.800831 at times  500
accuracy is 0.811918 at times  550
accuracy is 0.820036 at times  600
accuracy is 0.827757 at times  650
accuracy is 0.836468 at times  700
accuracy is 0.843001 at times  750
accuracy is 0.847951 at times  800
accuracy is 0.850327 at times  850
accuracy is 0.854682 at times  900
accuracy is 0.85983 at times  950


In [ ]:
train_writer.close()
test_writer.close()

In [ ]:
!pwd

# shell: 
```
tensorboard --logdir=mnist_logs/
```
# Browser: http://127.0.0.1:6006